# Оабораторная работа №2 по вычислительной математике

## Прменение прямых и итерационных методов для решения СЛАУ

### Выполнил Филиппенко Павел -- студент группы Б01-009

In [232]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
import math

##### Зададим нормы векторов

$$||x||_1 = max_i |x_i|$$

$$||x||_2 = \sum |x_i|$$

$$||x||_2 = (x, x)$$

In [233]:
def fst_vec_norm(x: np.ndarray):
    return max(abs(x))

def scd_vec_norm(x: np.ndarray):
    return sum(abs(x))

def trd_vec_norm(x: np.ndarray):
    return math.sqrt(np.dot(x, x))

##### Зададим нормы матриц

(по строкам)
$$||A||_1 = max_i \sum_j |a_{ij}|$$

(по столбцам)
$$||A||_2 = max_j \sum_i |a_{ij}|$$

$$||A||_3 = \sqrt{max_i \lambda_i(A^* A)}$$

Поскольку в данной работе мы рассмотриваем матрицы действительного пространства, $A^* = A^T$

In [234]:
def fst_m_norm(A: np.ndarray):
    assert(A.shape[0] == A.shape[1])
    return max([sum(abs(A[i])) for i in range(A.shape[0])])

def scd_m_norm(A: np.ndarray):
    assert(A.shape[0] == A.shape[1])
    return max([sum(abs(A.T[i])) for i in range(A.T.shape[0])])

# поскольку работаем в R, эрмитово сопряжение эквивалентвно транспонированию
def trd_m_norm(A: np.ndarray):
    B = np.dot(A.T, A)
    num, _ =  np.linalg.eigh(B)
    print(num)
    return math.sqrt(max(num))

##### Класс Slae представляет систему линейных уравнений.

_Поля_:
- A -- матрица системы
- f -- столбец решений

_Методы_:
- dimention -- декоратор, возвращающий порядок системы
- check_symmetric -- проверка матрицы на симметричность
- Gauss_mthd   -- решение системы линейных уравнений методом Гауса
- LU_mthd      -- решение системы линейных уравнений методом LU-разложения
- Holecky_mthd -- решение системы линейных уравнений методом Холецкого
- Zaydel_mthd  -- решение системы линейных уравнений методом Зейделя

In [235]:
# порядок округдения коэффициентов (нужен исключительно для вывода)
# при вычислениях коэффициенты НЕ ОКРУГЛЯЮТСЯ
round_n = 3

class Slae:
    def __init__(self, matrix: np.ndarray, values: np.ndarray):

        # проверяем, что матрица квадратная и вектор значений имеет соответсвующую размерность
        assert(matrix.shape[0] == matrix.shape[1])
        assert(matrix.shape[0] == values.shape[0])
        
        self.A = matrix
        self.f = values

    @property
    def dimention(self):
        return self.A.shape[0]

    def check_symmetric(self, a, tol=1e-16):
        return not False in (np.abs(self.A-self.A.T) < tol)

    def Gauss_mthd(self):
        # размерность матрицы
        n = self.dimention

        # прямой ход метода Гауса
        for k in range(n):
            for m in range(k+1, n):

                alpha = self.A[m][k] / self.A[k][k]

                self.f[m] = self.f[m] - self.f[k] * alpha 
                for i in range(k, n):
                    self.A[m][i] = self.A[m][i] - self.A[k][i] * alpha

        # обратный ход
        solution = np.full((n, ), 0.0)
        
        # поскольку индексы в python начинаются с 0 и заканчиваются n-1, последнее уравнение имеет индекс n-1
        solution[n-1] = self.f[n-1] / self.A[n-1][n-1]

        # предпоследнее уравнение имеет интекс n-2
        # поскольку функция range возвращает полуоткрытый интервал, вторым параметром ей передаеся -1, а не 0
        for k in range(n-2, 0-1, -1):
            solution[k] = 1 / self.A[k][k] * (self.f[k] - np.dot(self.A[k], solution))

        return solution

    def LU_mthd(self):
        pass

    def Holecky_mthd(self):
        pass

    def Zaydel_mthd(self):
        pass

    ## overloading output
    def __str__(self):
        n = self.dimention

        res = ''
        for i in range(n):
            string = ''
            for j in range(n):
                string = string + str(round(self.A[i][j], round_n)) + ' x{}'.format(j + 1)
                # string = string + str(self.A[i][j]) + ' x{}'.format(j + 1)
                if j != n - 1:
                    string = string + ' + '
                else:
                    string = string + ' = ' + str(round(self.f[i], round_n))
                    # string = string + ' = ' + str(self.f[i])
            string = string + '\n'
            res = res + string

        return res

##### Зададим систему уравнений через матрицу и столбей решений.

In [236]:
N = 12
A = np.eye(N)
f = np.full((N, ), 1.0)


for i in range(N):
    for j in range(N):
        if i == j:
            A[i][j] = 1
        else:
            A[i][j] = 1 / ((i+1)**2 + (j+2))
    f[i] = 1 / (i + 1)

# eq = Slae(A, f)